In [1]:
#include <chrono>
#include "src/tilemap.hpp"
#include "src/player.hpp"
#include "src/item.hpp"
#include "src/itemmanager.hpp"
#include "src/enemylist.hpp"

In [2]:
uint64_t timeSinceEpochMillisec() {
  using namespace std::chrono;
  return duration_cast<milliseconds>(system_clock::now().time_since_epoch()).count();
}

In [3]:
SVG canvas = SVG(1500, 1600, "CyberCrawler 2077");

TileMap tileMap = TileMap(&canvas);
EnemyList enemyList = EnemyList(&tileMap, &canvas);
Player player = Player(tileMap.getRandomFreeTile() -> getMapX(), tileMap.getRandomFreeTile() -> getMapY(), &tileMap, &enemyList, &canvas);

uint64_t t = timeSinceEpochMillisec();

ItemManager im = ItemManager(&player);
//Spawnt die Objekte in das Spielfeld (Itemart(1 = Leben, 2 = Score, 3, = ATK), Position im Feld(x, y), Zeiger aufs Spielfeld)
im.spawnItem(1, 5, 8, &canvas);
im.spawnItem(2, 4, 7, &canvas);
im.spawnItem(3, 3, 7, &canvas);
int itemPosX = 0; //Variabeln für die Zerstörung der Items
int itemPosY = 0;
bool isOnItem = false;

std::string key = canvas.lastKey();
while(key != "p") {
    key = canvas.lastKey();
    player.keyMovement(key, &enemyList);
    if (t + 1000 <= timeSinceEpochMillisec() && enemyList.getSize() != 0) { 
        for(int i = 0; i < enemyList.getSize(); i++) {
            player.setHP(player.getHP() - enemyList.getEnemy(i) -> autoMove());
        }
        t = timeSinceEpochMillisec();
    }
    //Für die Zerstörung von Items zuständig
    for(int i = 0; i < 3; i++){
        im.itemUpdate(&isOnItem, &itemPosX, &itemPosY, i);
    }
    if(isOnItem){
       im.deleteItem(itemPosX, itemPosY);
       isOnItem = false;
    }
}

ErrorErrorErrorErrorErrorErrorErrorErrorErrorErrorErrorErrorErrorErrorErrorErrorErrorError